In [1]:
import pandas as pd
import numpy as np
import os
import gc
from tqdm import tqdm_notebook # check the progressbar in the python. 
import glob # check the file name in fold. 

In [2]:
import matplotlib.pyplot as plt

In [ ]:
import pyspark

import findspark
findspark.init()
findspark.find()

In [ ]:
from pyspark import SparkContext

# C:\Users\User\Documents\R&D Challenge2019
path = 'C:/Users/User/Documents/R_D Challenge2019/Challenge19_GameBot_Preliminary/dataset/'
sc = SparkContext()

In [ ]:
import functools 

def unionAll(dfs):
    return functools.reduce(lambda df1,df2: df1.union(df2.select(df1.columns)), dfs) 

In [ ]:
from pyspark.sql import SQLContext
from pyspark import SparkFiles
from pyspark.sql.functions import lit
import pyspark.sql.functions as f

sqlContext = SQLContext(sc)
for i in tqdm_notebook(read_file_lst):
    # sqlContext.read.csv() : path를 C:Users/..의 형식으로 맞춰줘야함. (초기값이 spark설치된 곳으로 지정되어있음)
    df_temp = sqlContext.read.csv(SparkFiles.get(i), header=True, inferSchema= True) # read.csv 
    
    # transformation으로 추가하려면, df = df.withColumn("age_square", col("age")**2)
    df_temp = df_temp.withColumn("Date", lit(i[-8:-4])) # column 추가, lit명령어가 value를 추가하는 방법. 
    
    if i == read_file_lst[0]:
        df_total = df_temp
    else:
        df_total = unionAll([df_total, df_temp]) # row-wise 결합
        
    del df_temp 
    gc.collect()

In [ ]:
df_total = df_total.withColumnRenamed("actor_account", "account")

In [ ]:
outputpath = 'C:/Users/User/Documents/R_D Challenge2019/Challenge19_GameBot_Preliminary/preprocessing/'
df_total.filter(df_total['log_id'] == 126).select(['account', 'target_account']).toPandas().to_csv(outputpath + "party.csv", index=False)
df_total.filter(df_total['log_id'] == 134).select(['account', 'target_account']).toPandas().to_csv(outputpath + "friend.csv", index=False)
df_total.filter(df_total['log_id'] == 158).select(['account', 'target_account']).toPandas().to_csv(outputpath + "dual.csv", index=False)
df_total.filter(df_total['log_id'] == 229).select(['account', 'target_account']).toPandas().to_csv(outputpath + "mail.csv", index=False)
df_total.filter(df_total['log_id'] == 210).select(['account', 'target_account']).toPandas().to_csv(outputpath + "trade1.csv", index=False)
df_total.filter(df_total['log_id'] == 219).select(['account', 'target_account']).toPandas().to_csv(outputpath + "trade2.csv", index=False)
df_total.filter(df_total['log_id'] == 247).select(['account', 'target_account']).toPandas().to_csv(outputpath + "privateshop.csv", index=False)

In [4]:
# C:\Users\User\Documents\R&D Challenge2019
path = 'C:/Users/User/Documents/R_D Challenge2019/Challenge19_GameBot_Preliminary/preprocessing/'

In [5]:
dual = pd.read_csv(path + "dual.csv")
friend = pd.read_csv(path + "friend.csv")
mail = pd.read_csv(path + "mail.csv")
party = pd.read_csv(path + "party.csv")
privateshop = pd.read_csv(path + "privateshop.csv")
trade1 = pd.read_csv(path + "trade1.csv")
trade2 = pd.read_csv(path + "trade2.csv")

In [ ]:
# C:\Users\User\Documents\R&D Challenge2019
path = 'C:/Users/User/Documents/R_D Challenge2019/Challenge19_GameBot_Preliminary/'
train = pd.read_csv(path + 'labeled_accounts.csv')
test = pd.read_csv(path + 'test_accounts.csv')

In [ ]:
def network_features(train, test, my_graph, name_id):
    i = 0
    name = ["degree", "between_cc", "close_cc", "degree_cc",  "pagerank", "eigen_cc", "hub", "authorities"]
    degree = nx.degree(my_graph); 
    between_cc = nx.betweenness_centrality(my_graph); # 이것도 생각보다 오래걸림. 
    close_cc = nx.closeness_centrality(my_graph);
    degree_cc = nx.degree_centrality(my_graph);
    eigen_cc = nx.eigenvector_centrality_numpy(my_graph, max_iter=500);
    hub, authorities = nx.hits(my_graph);
    pagerank = nx.pagerank(my_graph, alpha=0.90); #PageRank applied
    
    
    for fe in [degree, between_cc, close_cc, degree_cc, pagerank, eigen_cc , hub, authorities]:
        fe = dict(fe) 
        fe_df = pd.DataFrame()
        fe_df['account'] = list(fe.keys())
        fe_df['{}_{}'.format(name_id, name[i])] = list(fe.values())
        i = i+1
        train = train.merge(fe_df,how='left',on='account')
        test = test.merge(fe_df,how='left',on='account')

    print("Network features maked")
    return train, test

In [ ]:
import networkx as nx
name_id = ["dual", "friend"]; num = 0
for df in tqdm_notebook([dual, friend, mail]):
    network_list = []
    for i in tqdm_notebook(np.arange(df.shape[0])):
        network_list.append((df.iloc[i]['account'], df.iloc[i]['target_account']))
    # Create a networkx graph object

    my_graph = nx.Graph() 
    my_graph.add_edges_from(network_list)
    train, test = network_features(train, test, my_graph, name_id[num])
    num += 1

In [ ]:
# C:\Users\User\Documents\R&D Challenge2019
outputpath = 'C:/Users/User/Documents/R_D Challenge2019/Challenge19_GameBot_Preliminary/features/'
train.to_csv(outputpath + "train_network1.csv", index=False)
test.to_csv(outputpath + "test_network1.csv", index=False)

In [ ]:
def network_features(train, test, my_graph, name_id):
    i = 0
    name = ["degree", "between_cc", "close_cc", "degree_cc",  "pagerank"]
    degree = nx.degree(my_graph); 
    between_cc = nx.betweenness_centrality(my_graph); # 이것도 생각보다 오래걸림. 
    close_cc = nx.closeness_centrality(my_graph);
    degree_cc = nx.degree_centrality(my_graph);
    #eigen_cc = nx.eigenvector_centrality_numpy(my_graph, max_iter=500);
    #hub, authorities = nx.hits(my_graph);
    pagerank = nx.pagerank(my_graph, alpha=0.90); #PageRank applied
    
    
    for fe in [degree, between_cc, close_cc, degree_cc, pagerank]:
        fe = dict(fe) 
        fe_df = pd.DataFrame()
        fe_df['account'] = list(fe.keys())
        fe_df['{}_{}'.format(name_id, name[i])] = list(fe.values())
        i = i+1
        train = train.merge(fe_df,how='left',on='account')
        test = test.merge(fe_df,how='left',on='account')

    print("Network features maked")
    return train, test

In [ ]:
# C:\Users\User\Documents\R&D Challenge2019
path = 'C:/Users/User/Documents/R_D Challenge2019/Challenge19_GameBot_Preliminary/'
train = pd.read_csv(path + 'labeled_accounts.csv')
test = pd.read_csv(path + 'test_accounts.csv')

In [ ]:
name_id = ['party', 'privateshop', 'trade1', 'trade2']; num = 0
for df in tqdm_notebook([party, privateshop, trade1, trade2]):
    network_list = []
    for i in tqdm_notebook(np.arange(df.shape[0])):
        network_list.append((df.iloc[i]['account'], df.iloc[i]['target_account']))
    # Create a networkx graph object

    my_graph = nx.Graph() 
    my_graph.add_edges_from(network_list)
    train, test = network_features(train, test, my_graph, name_id[num])
    num += 1

In [ ]:
# C:\Users\User\Documents\R&D Challenge2019
outputpath = 'C:/Users/User/Documents/R_D Challenge2019/Challenge19_GameBot_Preliminary/features/'
train.to_csv(outputpath + "train_network2.csv", index=False)
test.to_csv(outputpath + "test_network2.csv", index=False)